## Prequisites

### Before running the build script, ensure to have the following dependent packages

### 1. kfp
### 2. docker - should be logged in using docker login
### 3. yq - install using snap - https://github.com/mikefarah/yq


In [5]:
! ./build.sh pytorch_pipeline/examples/bert

++++ Building component images with tag=12-05-2021-18-05-41.207937
IMAGE TO BUILD: jagadeeshj/pytorch_pipeline:12-05-2021-18-05-41.207937
Sending build context to Docker daemon  3.723MB
Step 1/9 : FROM pytorch/pytorch:latest
 ---> 5ffed6c83695
Step 2/9 : COPY requirements.txt requirements.txt
 ---> 6e8bf822d04e
Step 3/9 : RUN apt-get update
 ---> Running in f192301ff752
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic InRelease [242 kB]
Get:3 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1411 kB]
Get:4 http://security.ubuntu.com/ubuntu bionic-security/multiverse amd64 Packages [24.7 kB]
Get:5 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2150 kB]
Get:6 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [423 kB]
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-backpor

In [22]:
# ! cat pytorch_bert.yaml

In [23]:
import kfp
import json
import os
from kfp import components
from kfp.components import load_component_from_file, load_component_from_url
from kfp import dsl
from kfp import compiler
from pathlib import Path

kfp.__version__

'1.4.0'

In [37]:
KFP_URL='istio-ingressgateway.istio-system.svc.cluster.local'
AUTH="authservice_session=MTYyMDgzOTU5MnxOd3dBTkV0SlZEWlBWRGRaUzFSUldsa3lSazFhUzA1TFExQXpSMUZDTjBKVVEwUlZUMDFaVmtsSVRsbE1RVUpCVXpRelFWbEtVMUU9fFyuc503Oxrz0CRLLSHWL6iu7egeWl57XpHXz1F9-kXf"
namespace="kubeflow-user-example-com"
client = kfp.Client(host=KFP_URL+"/pipeline", cookies=AUTH)
experiments = client.list_experiments(namespace=namespace)
my_experiment = experiments.experiments[0]
my_experiment

{'created_at': datetime.datetime(2021, 4, 22, 8, 44, 39, tzinfo=tzlocal()),
 'description': None,
 'id': 'aac96a63-616e-4d88-9334-6ca8df2bb956',
 'name': 'Default',
 'resource_references': [{'key': {'id': 'kubeflow-user-example-com',
                                  'type': 'NAMESPACE'},
                          'name': None,
                          'relationship': 'OWNER'}],
 'storage_state': 'STORAGESTATE_AVAILABLE'}

In [32]:
# Execute pipeline
run = client.run_pipeline(my_experiment.id, 'pytorch-bert', 'pytorch_bert.yaml')

In [38]:
HOST="bertserve.kubeflow-user-example-com.example.com"

In [40]:
!curl -v -H "Host: $HOST" -H "Cookie: $AUTH" "$KFP_URL/v1/models/bert:predict" -d @./examples/bert/sample.txt > ./bert_prediction_output.json

*   Trying 10.100.251.14:80...
* TCP_NODELAY set
* Connected to istio-ingressgateway.istio-system.svc.cluster.local (10.100.251.14) port 80 (#0)
> POST /v1/models/bert:predict HTTP/1.1
> Host: bertserve.kubeflow-user-example-com.example.com
> User-Agent: curl/7.68.0
> Accept: */*
> Cookie: authservice_session=MTYyMDgzOTU5MnxOd3dBTkV0SlZEWlBWRGRaUzFSUldsa3lSazFhUzA1TFExQXpSMUZDTjBKVVEwUlZUMDFaVmtsSVRsbE1RVUpCVXpRelFWbEtVMUU9fFyuc503Oxrz0CRLLSHWL6iu7egeWl57XpHXz1F9-kXf
> Content-Length: 84
> Content-Type: application/x-www-form-urlencoded
> 
* upload completely sent off: 84 out of 84 bytes
* Mark bundle as not supporting multiuse
< HTTP/1.1 200 OK
< content-length: 33
< content-type: application/json; charset=UTF-8
< date: Wed, 12 May 2021 18:53:21 GMT
< server: istio-envoy
< x-envoy-upstream-service-time: 476
< 
* Connection #0 to host istio-ingressgateway.istio-system.svc.cluster.local left intact
{"predictions": ["\"Sci/Tech\""]}

In [41]:
!curl -v -H "Host: $HOST" -H "Cookie: $AUTH" "$KFP_URL/v1/models/bert:explain" -d @./examples/bert/sample.txt  > bert_explaination_output.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0*   Trying 10.100.251.14:80...
* TCP_NODELAY set
* Connected to istio-ingressgateway.istio-system.svc.cluster.local (10.100.251.14) port 80 (#0)
> POST /v1/models/bert:explain HTTP/1.1
> Host: bertserve.kubeflow-user-example-com.example.com
> User-Agent: curl/7.68.0
> Accept: */*
> Cookie: authservice_session=MTYyMDgzOTU5MnxOd3dBTkV0SlZEWlBWRGRaUzFSUldsa3lSazFhUzA1TFExQXpSMUZDTjBKVVEwUlZUMDFaVmtsSVRsbE1RVUpCVXpRelFWbEtVMUU9fFyuc503Oxrz0CRLLSHWL6iu7egeWl57XpHXz1F9-kXf
> Content-Length: 84
> Content-Type: application/x-www-form-urlencoded
> 
} [84 bytes data]
* upload completely sent off: 84 out of 84 bytes
100    84    0     0  100    84      0      1  0:01:24  0:00:45  0:00:39     008     0* Mark bundle as not supporting multiuse
< HTTP/1.1 200 OK
< cont

In [42]:
import json
explanations_json = json.loads(open("./bert_explaination_output.json", "r").read())
explanations_json

{'explanations': [{'words': ['[CLS]',
    'bloomberg',
    'has',
    'reported',
    'on',
    'the',
    'economy',
    '[SEP]'],
   'importances': [0.49803317807827413,
    -0.04228915625436579,
    -0.22691037181108395,
    0.15573719339552444,
    0.08677259891698845,
    0.1791962203959244,
    0.525546079847318,
    -0.5988261343532961],
   'delta': 0.12081549835977756}]}

In [43]:
prediction_json = json.loads(open("./bert_prediction_output.json", "r").read())

In [44]:
import torch
attributions = explanations_json["explanations"][0]['importances']
tokens = explanations_json["explanations"][0]['words']
delta = explanations_json["explanations"][0]['delta']

attributions = torch.tensor(attributions)
pred_prob = 0.75
pred_class = prediction_json["predictions"][0]
true_class = "Business"
attr_class ="world"

In [45]:
from captum.attr import visualization
vis_data_records =[]
vis_data_records.append(visualization.VisualizationDataRecord(
                            attributions,
                            pred_prob,
                            pred_class,
                            true_class,
                            attr_class,
                            attributions.sum(),       
                            tokens,
                            delta))

In [46]:
visualization.visualize_text(vis_data_records)

True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
Business,"""Sci/Tech"" (0.75)",world,0.58,[CLS] bloomberg has reported on the economy [SEP]


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
Business,"""Sci/Tech"" (0.75)",world,0.58,[CLS] bloomberg has reported on the economy [SEP]
